In [ ]:
# ============================================
# Chunked text exporter (CHAT+SPEECH / CHAT-only)
# - 時間表記はテキストに含めない（[SOURCE] 本文 のみ）
# - 10分チャンクで分割し、GCSへ逐次アップロード
# - 進捗バー表示、再実行時は既存ファイルを検出してスキップ（レジューム対応）
# ============================================

from __future__ import annotations
import os
import re
from typing import List, Dict, Optional, Iterable, Tuple

import pandas as pd
from tqdm.auto import tqdm

# GCS
import gcsfs

# -----------------------
# 設定
# -----------------------
# GCS 上の入力ディレクトリ（コメントCSVが大量にある場所）
COMMENT_DIR_GCS = "gs://dena-ai-intern-yoshihara-data/pococha_comment_sorted"

# 全 live_id を俯瞰できる “一覧CSV” がローカルにある場合は指定（なければ自動でGCS列挙）
# 例: 前段のリストアップ処理で作った pococha_comment_sorted_suffixes.csv
LOCAL_LISTING_CSV = "pococha_comment_sorted_suffixes.csv"  # 無ければ自動列挙にフォールバック

# 音声（文字起こし）全集（live_id混在）CSVのGCSパス
AUDIO_GCS_PATH = "gs://dena-ai-intern-yoshihara-data/pococha_speech/bq-results-20250903-061249-1756880068827.csv"

# 出力先（拡張子は .txt を採用）
OUT_COMBINED_PREFIX = "gs://dena-ai-intern-yoshihara-data/yoshi_LLMQA_comment_speeech_combined"  # ←speeech（eが3つ）仕様
OUT_CHATONLY_PREFIX = "gs://dena-ai-intern-yoshihara-data/yoshi_LLMQA_comment_only"
OUT_EXT = ".txt"

# 10分 = 600秒 チャンク
CHUNK_SECONDS = 600

# 例外が出た live_id を記録して最後に表示する
ERROR_LOG: List[Tuple[int, str]] = []

# --------------------------------
# ユーティリティ
# --------------------------------
def parse_live_id_from_comment_path(path: str) -> Optional[int]:
    """
    パス末尾の comment_68412208.csv から live_id を抽出
    """
    base = os.path.basename(path)
    m = re.search(r"comment_(\d+)\.csv$", base)
    return int(m.group(1)) if m else None


def to_int_seconds_safe(x) -> int:
    """
    秒の列を安全にint化（NaN→0、負値→0）
    """
    v = pd.to_numeric(x, errors="coerce")
    v = 0 if pd.isna(v) else int(v)
    return max(0, v)


def list_comment_files(fs: gcsfs.GCSFileSystem, comment_dir_gcs: str, local_listing_csv: Optional[str]) -> List[str]:
    """
    コメントCSV（comment_*.csv）のGCSパスを列挙。
    可能ならローカルの一覧CSV（suffix_number付き）を使い、なければgcsfsで列挙。
    """
    if local_listing_csv and os.path.exists(local_listing_csv):
        df = pd.read_csv(local_listing_csv)
        if "gcs_path" in df.columns:
            files = df["gcs_path"].dropna().astype(str).tolist()
            # 念のためフィルタ
            files = [p for p in files if p.startswith(comment_dir_gcs) and p.endswith(".csv")]
            return files

    # Fallback: GCSを直接列挙
    pattern = comment_dir_gcs.rstrip("/") + "/comment_*.csv"
    return fs.glob(pattern)


def read_comments_one_live(fs: gcsfs.GCSFileSystem, comment_csv_gcs: str) -> pd.DataFrame:
    """
    単一 live_id のコメントCSVを読み、必要列があるかバリデート。
    必須列: ['live_id','user_id','text','comment_time','live_started_time','elapsed_seconds']
    """
    df = pd.read_csv(comment_csv_gcs)
    required = ['live_id', 'user_id', 'text', 'comment_time', 'live_started_time', 'elapsed_seconds']
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"コメントCSVに必要な列が不足: {missing} @ {comment_csv_gcs}")
    # 整形
    out = df.copy()
    out["t"] = [to_int_seconds_safe(x) for x in out["elapsed_seconds"]]
    out["source"] = "CHAT"
    out["content"] = out["text"].astype(str).fillna("").str.strip()
    out["orig_idx"] = range(len(out))
    return out[["live_id","source","t","content","orig_idx"]]


def read_audio_all(fs: gcsfs.GCSFileSystem, audio_csv_gcs: str) -> pd.DataFrame:
    """
    音声（文字起こし）全集を読み込み（live_id混在）。
    必須列: ['live_id','audio_path','timestamp_start','timestamp_end','transcription']
    """
    df = pd.read_csv(audio_csv_gcs)
    required = ['live_id', 'audio_path', 'timestamp_start', 'timestamp_end', 'transcription']
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"音声CSVに必要な列が不足: {missing} @ {audio_csv_gcs}")

    # t=timestamp_start(int秒)に正規化
    out = df.copy()
    out["t"] = [to_int_seconds_safe(x) for x in out["timestamp_start"]]
    out["source"] = "SPEECH"
    out["content"] = out["transcription"].astype(str).fillna("").str.strip()
    out["orig_idx"] = range(len(out))
    return out[["live_id","source","t","content","orig_idx"]]


def merge_chat_speech(chat_df: pd.DataFrame, speech_df: pd.DataFrame) -> pd.DataFrame:
    """
    CHATとSPEECHを時間順にマージ（安定ソート）。
    タイブレーク: t昇順、CHAT先→SPEECH後、orig_idx順維持。
    """
    # 空コンテンツ除去
    chat = chat_df[chat_df["content"].astype(str).str.len() > 0].copy()
    speech = speech_df[speech_df["content"].astype(str).str.len() > 0].copy()

    merged = pd.concat([chat, speech], ignore_index=True)
    source_order = merged["source"].map({"CHAT":0, "SPEECH":1}).fillna(2).astype(int)
    merged = merged.assign(_source_order=source_order)
    merged = merged.sort_values(["t","_source_order","orig_idx"], kind="mergesort").drop(columns=["_source_order"])
    merged = merged.reset_index(drop=True)
    return merged


def iter_chunk_indices(max_t: int, chunk_seconds: int = CHUNK_SECONDS) -> Iterable[int]:
    """
    最大時刻（秒）から、1始まりのチャンク番号を列挙。
    例: max_t=0..599 -> 1のみ, 600..1199 -> 1,2
    """
    if max_t < 0:
        return []
    last_chunk = (max_t // chunk_seconds) + 1 if max_t >= 0 else 1
    return range(1, last_chunk + 1)


def build_text_lines(df: pd.DataFrame) -> List[str]:
    """
    行を [SOURCE] 本文 の形式にする（時間は含めない）。
    ※ f-string の {expr} 内にバックスラッシュを含むと SyntaxError になるので、
       事前にクレンジングしてから f-string で変数を埋め込む。
    """
    if df.empty:
        return []
    lines: List[str] = []
    for row in df.itertuples(index=False):
        # 改行をスペースに、前後空白トリム
        text_clean = str(row.content).replace("\n", " ").strip()
        line = f"[{row.source}] {text_clean}"
        lines.append(line)
    return lines


def write_text_gcs(fs: gcsfs.GCSFileSystem, gcs_path: str, text: str) -> None:
    """
    GCSへUTF-8でテキスト書き込み（上書き）。
    """
    with fs.open(gcs_path, "w", encoding="utf-8") as f:
        f.write(text if text.endswith("\n") else text + "\n")


def ensure_prefix_slash_removed(prefix: str) -> str:
    return prefix[:-1] if prefix.endswith("/") else prefix


def output_path(prefix: str, live_id: int, chunk_idx: int, ext: str = OUT_EXT) -> str:
    prefix = ensure_prefix_slash_removed(prefix)
    return f"{prefix}/{live_id}_{chunk_idx}{ext}"


# --------------------------------
# メイン処理
# --------------------------------
def main():
    fs = gcsfs.GCSFileSystem()

    # 1) 対象コメントファイルの一覧を取得
    comment_files = list_comment_files(fs, COMMENT_DIR_GCS, LOCAL_LISTING_CSV)
    if not comment_files:
        print("コメントCSVが見つかりません。設定や権限を確認してください。")
        return

    # 2) 音声全集を一度だけ読み込み
    print("音声CSVを読込中...")
    audio_all = read_audio_all(fs, AUDIO_GCS_PATH)

    # live_id -> speech_df のビューをすぐ作れるように groupby
    audio_grouped = audio_all.groupby("live_id", sort=False)

    # 3) 進捗バー（外側：ライブ数）
    created_count = 0
    skipped_count = 0

    # live_idを安定順で処理（path上の番号順）
    comment_files_sorted = sorted(comment_files, key=lambda p: parse_live_id_from_comment_path(p) or 0)

    pbar = tqdm(comment_files_sorted, desc="Lives", unit="live")

    for comment_csv in pbar:
        try:
            live_id = parse_live_id_from_comment_path(comment_csv)
            if live_id is None:
                # 形式外のファイルはスキップ
                continue

            pbar.set_postfix_str(f"live_id={live_id}")

            # コメント読み込み
            chat_df = read_comments_one_live(fs, comment_csv)
            # 該当liveの音声（無い可能性がある）
            try:
                speech_df = audio_grouped.get_group(live_id)[["live_id","source","t","content","orig_idx"]].copy()
            except KeyError:
                speech_df = pd.DataFrame(columns=["live_id","source","t","content","orig_idx"])

            # マージ（時間順）
            merged = merge_chat_speech(chat_df, speech_df)

            # チャンク番号付与（1始まり）
            if not merged.empty:
                merged["chunk_idx"] = (merged["t"] // CHUNK_SECONDS) + 1
            if not chat_df.empty:
                chat_df = chat_df.sort_values(["t","orig_idx"], kind="mergesort").reset_index(drop=True)
                chat_df["chunk_idx"] = (chat_df["t"] // CHUNK_SECONDS) + 1

            # max_t を把握してチャンク列挙（両データの最大）
            max_t_candidates = []
            if not merged.empty:
                max_t_candidates.append(int(merged["t"].max()))
            if not chat_df.empty:
                max_t_candidates.append(int(chat_df["t"].max()))
            max_t = max(max_t_candidates) if max_t_candidates else -1

            if max_t < 0:
                # 何も無ければスキップ
                continue

            # 内側の進捗バー（このliveで出力する総チャンク数×2（combined/chatonly））
            live_chunks = list(iter_chunk_indices(max_t, CHUNK_SECONDS))
            inner_total = len(live_chunks) * 2  # combined + chatonly
            inner_bar = tqdm(total=inner_total, desc=f"live {live_id}", leave=False)

            # ---- Combined（CHAT+SPEECH） ----
            for idx in live_chunks:
                out_path = output_path(OUT_COMBINED_PREFIX, live_id, idx, OUT_EXT)
                if fs.exists(out_path):
                    skipped_count += 1
                    inner_bar.update(1)
                    inner_bar.set_postfix_str(f"combined skip {os.path.basename(out_path)}")
                    continue

                # チャンク内の行を抽出（時系列保持済み）
                if merged.empty:
                    lines = []
                else:
                    part = merged[merged["chunk_idx"] == idx]
                    lines = build_text_lines(part)

                # 空チャンクはスキップ（ファイルを作らない）
                if not lines:
                    inner_bar.update(1)
                    inner_bar.set_postfix_str(f"combined empty chunk {idx}")
                    continue

                # 書き込み
                write_text_gcs(fs, out_path, "\n".join(lines))
                created_count += 1
                inner_bar.update(1)
                inner_bar.set_postfix_str(f"combined wrote {os.path.basename(out_path)}")

            # ---- Chat-only（SPEECH抜き） ----
            for idx in live_chunks:
                out_path = output_path(OUT_CHATONLY_PREFIX, live_id, idx, OUT_EXT)
                if fs.exists(out_path):
                    skipped_count += 1
                    inner_bar.update(1)
                    inner_bar.set_postfix_str(f"chatonly skip {os.path.basename(out_path)}")
                    continue

                if chat_df.empty:
                    lines = []
                else:
                    part = chat_df[chat_df["chunk_idx"] == idx]
                    lines = build_text_lines(part)  # [CHAT] 本文 形式

                if not lines:
                    inner_bar.update(1)
                    inner_bar.set_postfix_str(f"chatonly empty chunk {idx}")
                    continue

                write_text_gcs(fs, out_path, "\n".join(lines))
                created_count += 1
                inner_bar.update(1)
                inner_bar.set_postfix_str(f"chatonly wrote {os.path.basename(out_path)}")

            inner_bar.close()

        except Exception as e:
            ERROR_LOG.append((parse_live_id_from_comment_path(comment_csv) or -1, str(e)))
            # エラーでも他liveは続行
            continue

    pbar.close()

    print("\n=== 完了レポート ===")
    print(f"作成: {created_count}  / スキップ: {skipped_count}")
    if ERROR_LOG:
        print(f"\nエラー {len(ERROR_LOG)} 件（一部抜粋）:")
        for live_id, msg in ERROR_LOG[:20]:
            print(f"  live_id={live_id}: {msg}")


# 実行
main()
